# Dreambooth
### Notebook implementation by Joe Penna (@MysteryGuitarM on Twitter) - Improvements by David Bielejeski
https://github.com/JoePenna/Dreambooth-Stable-Diffusion

### If on runpod / vast.ai / etc, spin up an A6000 or A100 pod using a Stable Diffusion template with Jupyter pre-installed.

## Build Environment

In [ ]:
# If running on Vast.AI, copy the code in this cell into a new notebook. Run it, then launch the `dreambooth_runpod_joepenna.ipynb` notebook from the jupyter interface.
!git clone https://github.com/JoePenna/Dreambooth-Stable-Diffusion

In [ ]:
%cd Dreambooth-Stable-Diffusion

#BUILD ENV
%pip install omegaconf
%pip install einops
%pip install pytorch-lightning==1.6.5
%pip install protobuf==3.20.1
%pip install test-tube
%pip install transformers
%pip install kornia
%pip install -e git+https://github.com/CompVis/taming-transformers.git@master#egg=taming-transformers
%pip install -e git+https://github.com/openai/CLIP.git@main#egg=clip
%pip install setuptools==59.5.0
%pip install pillow==9.0.1
%pip install torchmetrics==0.6.0
%pip install -e .
%pip install gdown
%pip install pydrive


In [ ]:
## DOWNLOAD SD
%pip install gdown

#Model hosted by David Bielejeski
!gdown https://drive.google.com/uc?id=1SXIecuTKoUTrPBh2Y95NiEtBBH9PbGAe

# Regularization Images

Training teaches your new model both your token **but** re-trains your class simultaneously.

From cursory testing, it does not seem like reg images affect the model too much. However, they do affect your class greatly, which will in turn affect your generations.

In [ ]:
#GENERATE 400 images
!python scripts/stable_txt2img.py --seed 10 --ddim_eta 0.0 --n_samples 1 --n_iter 400 --scale 10.0 --ddim_steps 50  --ckpt model.ckpt \
--prompt "person"

# If you don't want to train against "person", change it to whatever you want above, and on some of the cells below:

In [ ]:
# zip up the files for downloading and reuse.
!apt-get install -y zip
!zip -r all_images.zip outputs/

## Upload existing regularization images (200+ is recommended)

# grab the files here: https://github.com/djbielejeski/Stable-Diffusion-Regularization-Images
!git clone https://github.com/JoePenna/Stable-Diffusion-Regularization-Images

!mkdir outputs
!mkdir outputs/txt2img-samples
!mkdir outputs/txt2img-samples/samples

# RECOMMENDED - use person_ddim images (1500) - ddim @ 50 steps, CFG 7.0
!mv -v Stable-Diffusion-Regularization-Images/person_ddim outputs/txt2img-samples outputs/txt2img-samples/samples

# Use man_euler images
# provided by Niko Pueringer (Corridor Digital) - euler @ 40 steps, CFG 7.5
# !mv -v Stable-Diffusion-Regularization-Images/man_euler outputs/txt2img-samples outputs/txt2img-samples/samples

# Use man_unsplash images - pictures from various photographers
# !mv -v Stable-Diffusion-Regularization-Images/man_unsplash outputs/txt2img-samples outputs/txt2img-samples/samples

# Use woman_dimm images
# provided by David Bielejeski - ddim
# !mv -v Stable-Diffusion-Regularization-Images/woman_dimm outputs/txt2img-samples

# Upload your training images
Upload 10-20 images of someone to /workspace/Dreambooth-Stable-Diffusion/training_samples

WARNING: Be sure to upload an *even* amount of images, otherwise the training inexplicably stops at 1500 steps.

*   2-3 full body
*   3-5 upper body 
*   5-12 close-up on face

In [ ]:
# or upload from your Google Drive
%pip install gdown
!gdown https://drive.google.com/uc?id={fileId}
!mv {name of file} training_samples/{name of file}

## Prep Training Variables

Navigate to:
/workspace/Dreambooth-Stable-Diffusion/ldm/data/personalized.py

Change `demoura` in line 11 to whatever you want your token to be.

(Note: The original repo uses `sks` as a token -- but that has a relatively high correlation with the SKS semi-automatic rifle, which affects low-step generations)

e.g. I changed mine to
```python
training_templates_smallest = [
    'joepenna {}',
]
```

The last line of this file trains for `3000` steps.
/workspace/Dreambooth-Stable-Diffusion/configs/stable-diffusion/v1-finetune_unfrozen.yaml

If training a person or subject, keep an eye on your project's `/images/train/samples_scaled_gs-00xxxx` generations.

If training a style, keep an eye on your project's `/images/train/samples_gs-00xxxx` generations.

## Start Training (you should also change the code below)

In [ ]:
# START THE TRAINING
!python "main.py" \
 --base configs/stable-diffusion/v1-finetune_unfrozen.yaml \
 -t \
 --actual_resume "model.ckpt" \
 --reg_data_root "/workspace/Dreambooth-Stable-Diffusion/outputs/txt2img-samples/samples" \
 -n "project_name" \
 --gpus 0, \
 --data_root "/workspace/Dreambooth-Stable-Diffusion/training_samples" \
 --class_word "person" # << match this word to the class word from regularization images above

## Pruning (12GB to 2GB)

In [ ]:
# This will prune around 10GB from the ckpt file, then move it the "trained_models" folder
!python "scripts/prune-ckpt.py" --ckpt logs/{training_name}/checkpoints/last.ckpt
!mkdir trained_models
!mv logs/{training_name}/checkpoints/last-pruned.ckpt workspace/Dreambooth-Stable-Diffusion/trained_models/{training_name}.ckpt

## Generate Images With Your Trained Model!

In [ ]:
!python scripts/stable_txt2img.py \
--ddim_eta 0.0 --n_samples 1 --n_iter 4 \
--scale 7.0 --ddim_steps 50 --ckpt "/workspace/Dreambooth-Stable-Diffusion/logs/{training_name}/last.ckpt" \
--prompt "demoura person as a masterpiece portrait painting by John Singer Sargent in the style of Rembrandt"

In [ ]:
#Download the .ckpt file and use in your favorite Stable Diffusion repo!